In [52]:
import tifffile
import numpy as np
import pandas as pd
import cv2
from scipy.ndimage import center_of_mass, extrema
import napari 
import matplotlib.pyplot as plt
from skimage import measure

In [53]:
scalling_x = 22
scalling_y = 20
translation_x = 200 # 422.5
translation_y = 200 # 336

# load function and image

In [38]:
def get_coor (path_to_log_file,rescaleX,rescaleY):
    with open(path_to_log_file, "r", encoding='utf-8-sig') as f:
        string = f.read()
        lines = string.split("\n")
        num = 1
        event_num = []
        x_coordinates = []
        y_coordinates = []
        for line in lines:

            if len(line ) != 0:
                if (int(num) - 1)% 3  == 0: 
                    event_num.append(num)
                    num+=1
                
                else: 
                    y = line[1:-1].split(',')[1]
                    x = line[1:-1].split(',')[0]
                    y_coordinates.append((rescaleY * (-float(y)))+translation_x-scalling_y)
                    x_coordinates.append((rescaleX * float(x))+ translation_y-scalling_x)
                    #note: with center simulation Y -20 X -35
                    # 
                    num+=1
    return len(event_num) , pd.DataFrame({'y_coordinates': y_coordinates, 'x_coordinates': x_coordinates}) 

In [39]:
# Load tiff file and log position
im_path = "D:/Internship/NTU/simulation/with_center/multi_center_resized.ome.tif"
im = tifffile.imread(im_path)
rescaling_y = im.shape[1]/400
rescaling_x = im.shape[2]/400
rescaling_y,rescaling_x

(1.0, 1.0)

In [40]:
event_all_log = []
with open("D:/Internship/NTU/simulation/with_center/nonzero_frames_final.log", "r", encoding='utf-8-sig') as f:
    string = f.read()
    lines = string.split("\n")
    for line in lines:
        if len(line ) != 0:
            events = line.split(":")[1].split(",")
            frame = line.split(":")[0].split()[1]
           
            if len(events) == 1 :
                continue
            else:
                num_events = [x.split("=")[1] for x in events]
                x= int(num_events[0]) + float(num_events[1])
                num = [float(frame),x,float(num_events[2])]
                event_all_log.append(num)

In [41]:
# frame , fusion, fission 
event_all_log = (np.transpose(np.array(event_all_log)))
print(f"all_fusion {np.sum(event_all_log[1])}")
print(f"all_fission {np.sum(event_all_log[2])}")


all_fusion 62.0
all_fission 61.0


In [42]:
event_all_log

array([[  1.,   4.,   6.,   7.,  11.,  12.,  14.,  15.,  16.,  17.,  18.,
         19.,  20.,  21.,  22.,  23.,  25.,  27.,  28.,  29.,  33.,  34.,
         35.,  36.,  37.,  40.,  41.,  42.,  43.,  44.,  47.,  48.,  49.,
         52.,  54.,  55.,  56.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
         65.,  67.,  69.,  71.,  72.,  73.,  74.,  75.,  76.,  78.,  79.,
         80.,  82.,  85.,  86.,  87.,  88.,  90.,  91.,  92.,  93.,  94.,
         95.,  97.,  99., 100.],
       [  1.,   0.,   1.,   1.,   0.,   1.,   1.,   0.,   1.,   1.,   2.,
          0.,   1.,   1.,   1.,   1.,   1.,   0.,   1.,   3.,   1.,   1.,
          2.,   0.,   1.,   0.,   1.,   2.,   0.,   2.,   2.,   0.,   1.,
          1.,   1.,   1.,   1.,   0.,   1.,   0.,   1.,   1.,   1.,   1.,
          1.,   1.,   1.,   0.,   0.,   1.,   0.,   1.,   2.,   0.,   0.,
          1.,   1.,   2.,   1.,   0.,   2.,   1.,   0.,   2.,   1.,   1.,
          1.,   1.,   1.,   0.],
       [  0.,   1.,   0.,   0.,   1.,   0.,   

In [43]:
num_fission,fission_coor = get_coor("D:/Internship/NTU/simulation/with_center/fission_position_2.log", rescaling_x, rescaling_y)
num_fusion,fusion_coor = get_coor("D:/Internship/NTU/simulation/with_center/fusion_position_2.log", rescaling_x, rescaling_y)

print(f"num_fusion {num_fusion}")
print(f"num_fission {num_fission}")
fusion_coor


num_fusion 60
num_fission 61


,y_coordinates,x_coordinates
0,176.96922,184.39292
1,176.53138,188.25030
2,187.44330,206.21540
3,190.75630,208.37380
4,192.30460,175.07611
...,...,...
115,271.65960,163.81670
116,193.00420,211.35980
117,196.92170,213.30460
118,242.49980,241.77250


In [44]:
# append frame in the coordinate array. 
# each frame consistingg of at least 2 coordinates, one for each fusion and fission event
fusion_frames = []
fission_frames = []

fusion_idx = 0
fission_idx = 0

for x in range(len(event_all_log[0])):
    if event_all_log[1][x] > 0 :
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[1][x]*2 )
        if x == 0:
            fusion_frames = r_frame
        else:
            fusion_frames = np.concatenate((fusion_frames,r_frame))
        fusion_idx += 1
    if event_all_log[2][x] > 0:
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[2][x]*2 )

        if x == 0:
            fission_frames = r_frame
        else:
            fission_frames = np.concatenate((fission_frames,r_frame))

        fission_idx += 1

# Assign the lists as new columns in your DataFrames
fusion_coor['frame'] = fusion_frames[:-4]
fission_coor['frame'] = fission_frames

In [45]:
# need 2 loop coz frame are different
fusion_coords = []
for f in range(len(fusion_coor['frame'])):
    frame = int(fusion_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fusion_coor.iloc[f,0], fusion_coor.iloc[f,1]) # x,y coordinates or yx?
    fusion_coords.append(np.append(frame, coor))

fission_coords = []
for f in range(len(fission_coor['frame'])):
    frame = int(fission_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fission_coor.iloc[f,0], fission_coor.iloc[f,1]) # x,y coordinates or yx?
    fission_coords.append(np.append(frame, coor))


In [46]:
fusion_coor

,y_coordinates,x_coordinates,frame
0,176.96922,184.39292,1.0
1,176.53138,188.25030,1.0
2,187.44330,206.21540,6.0
3,190.75630,208.37380,6.0
4,192.30460,175.07611,7.0
...,...,...,...
115,271.65960,163.81670,93.0
116,193.00420,211.35980,94.0
117,196.92170,213.30460,94.0
118,242.49980,241.77250,95.0


In [47]:
fusion_coords

[array([  1.     , 176.96922, 184.39292]),
 array([  1.     , 176.53138, 188.2503 ]),
 array([  6.    , 187.4433, 206.2154]),
 array([  6.    , 190.7563, 208.3738]),
 array([  7.     , 192.3046 , 175.07611]),
 array([  7.     , 192.7606 , 172.50951]),
 array([ 12.    , 192.9082, 144.0649]),
 array([ 12.    , 191.6724, 144.8839]),
 array([ 14.    , 244.0259, 235.0287]),
 array([ 14.    , 241.6562, 233.4411]),
 array([ 16.     , 188.02152, 150.0056 ]),
 array([ 16.     , 185.97183, 145.3298 ]),
 array([ 17.    , 149.3605, 236.1426]),
 array([ 17.    , 146.0339, 232.8476]),
 array([ 18.     , 197.0191 , 171.84889]),
 array([ 18.     , 201.2549 , 171.67282]),
 array([ 18.    , 112.7768, 210.9044]),
 array([ 18.    , 108.4946, 210.9552]),
 array([ 20.    , 167.2587, 158.409 ]),
 array([ 20.    , 169.264 , 156.6836]),
 array([ 21.    , 167.2587, 158.409 ]),
 array([ 21.    , 162.8807, 156.1859]),
 array([ 22.    , 203.2423, 223.2977]),
 array([ 22.    , 206.2685, 226.1662]),
 array([ 23.    

In [48]:
fission_coor

,y_coordinates,x_coordinates,frame
0,187.81628,206.07220,4.0
1,191.12010,208.20900,4.0
2,111.77230,210.61550,11.0
3,107.61330,210.09190,11.0
4,239.03820,236.05750,14.0
...,...,...,...
117,238.94210,239.69310,99.0
118,191.85450,143.72230,100.0
119,187.77895,145.01420,100.0
120,247.78230,170.03168,100.0


# run visualization

In [49]:
middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    middle = np.vstack((middle, new_row))

In [118]:
'''
middle = np.array([1, 200, 200])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, 200, 200])[np.newaxis, :]
    middle = np.vstack((middle, new_row))
'''


'\nmiddle = np.array([1, 200, 200])[np.newaxis, :]  # shape (1, 3)\nfor i in range(1, im.shape[0]):\n    new_row = np.array([i+1, 200, 200])[np.newaxis, :]\n    middle = np.vstack((middle, new_row))\n'

In [50]:
# Y is in the first column and X is in the second. 
test_coord = [np.array([2,0,0]), np.array([2,422.5,0]), np.array([2,845,0]), 
              np.array([2,0,336]), np.array([2,0,672]), np.array([2,422.5,336]),np.array([  2    , 302.5515, 404.3512])]
label = [0,1,2,3,4,5,9] 

In [51]:
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =5, name = "central point")
viewer.add_points(fusion_coords, face_color='yellow', size =5, name = "fussion")
viewer.add_points(fission_coords, face_color='green', size =5, name = "fission")
viewer.add_points(test_coord, face_color='yellow', size =1 , text = label, name = "label number")
napari.run()

#TODO is it the correct position of this iamghe

In [121]:
'''
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =5, name = "label number")
viewer.add_points(test_coord, face_color='yellow', size =1 , text = label, name = "label number")
napari.run()
'''


'\n#show image\nviewer = napari.Viewer()\nviewer.add_labels(im, name ="labeled image")\nviewer.add_points(middle, face_color=\'white\', size =5, name = "label number")\nviewer.add_points(test_coord, face_color=\'yellow\', size =1 , text = label, name = "label number")\nnapari.run()\n'